In [43]:
import numpy as np
import pandas as pd
import seaborn as sns
from collections import defaultdict
from unidecode import unidecode
import matplotlib.pyplot as plt
plt.rcParams['svg.fonttype'] = 'none'

In [44]:
d = pd.read_csv('./barcode_results.tsv', sep="\t")
m = pd.read_csv('./sample_metadata.csv')
barcodes = [b for b in d.columns if b not in ['Sample','Dummy',]]
m['Strain_library'] = m['Library'].astype(str) + m['Strain_name']

In [45]:
negs = d[d.Sample.isin(m[m.Strain_name.str.contains("negative")]['Sample'])].set_index("Sample") +1 
max_dummy_ratio_in_negs = negs.div(negs.Dummy, axis=0).max().max()
print(max_dummy_ratio_in_negs)

2.142857142857143


In [46]:
zscores = defaultdict(list)

for index, row in d.iterrows():

    
    sample_data =row[barcodes + ["Dummy"]] + 1
    sample_sum = sample_data.sum()
    
    for barcode in barcodes:
        zscores['Sample'].append(row['Sample'])
        zscores['Barcode'].append(barcode)
        zscores['count'].append(row[barcode])
        zscores['ratio_to_dummy'].append(row[barcode] / (1+row['Dummy']))
        
zscores = pd.DataFrame(zscores)

final_ratios = zscores.pivot(index='Sample',columns='Barcode',values='ratio_to_dummy')

print(round((final_ratios ==  0).sum().sum() / (final_ratios.shape[0]*final_ratios.shape[1]),2))

final_ratios[final_pvalue > 0.01] = 0 # P value cutoff
print(round((final_ratios ==  0).sum().sum() / (final_ratios.shape[0]*final_ratios.shape[1]),2))

final_ratios[final_zscore < 0] = 0 # Only positive with respect to dummy
print(round((final_ratios ==  0).sum().sum() / (final_ratios.shape[0]*final_ratios.shape[1]),2))

# final_ratios[final_ratios < max_dummy_ratio_in_negs*5] = 0 # Minimum ratio
print(round((final_ratios ==  0).sum().sum() / (final_ratios.shape[0]*final_ratios.shape[1]),2))

final_ratios = round(final_ratios,1)

final = final_ratios.melt(ignore_index=False).reset_index()
final = final[~final.Sample.isin(negs.index)]

0.31
0.59
0.85
0.85


In [110]:
plasmid_meta = pd.read_csv('./plasmid_metadata.csv')
pgl0 = {}
pgl2 = {}
for index, row in plasmid_meta.iterrows():
    pgl0[row['ORI']] = row['pGL0']
    ori_lib = row['ORI'] + "-" + str(row['Library'])
    pgl2[ori_lib] = row['pGL2']

In [111]:
strain_meta = pd.read_csv('./strain_metadata.csv')
strain = {}
library = {}
for index, row in strain_meta.iterrows():
    strain[row['Sample']] = row['CVM']
    library[row['Sample']] = row['Library']

# Library 150

In [112]:
output_table = defaultdict(list)

for index, row in final.iterrows():
    ## get CVM
    if row['Sample'] in strain:
        if library[row['Sample']] == 150:
            cvm = strain[row['Sample']]
            p0 = pgl0[row['Barcode']]
            p2 = pgl2[row['Barcode'] + "-" + str(library[row['Sample']])]
            ## get pGL0
            output_table['Strain'].append(cvm)
            output_table['ORI (pGL0)'].append(p0)
            output_table['Plasmid (pGL2)'].append(p2)
            output_table['Fold enrichment'].append(row['value'])
    ## get pGL2
output_table = pd.DataFrame(output_table)
output_table.to_csv("ingest_for_portal150.csv")

# Library 147

In [87]:
output_table = defaultdict(list)

for index, row in final.iterrows():
    ## get CVM
    if row['Sample'] in strain:
        if library[row['Sample']] == 147:
            cvm = strain[row['Sample']]
            p0 = pgl0[row['Barcode']]
            p2 = pgl2[row['Barcode'] + "-" + str(library[row['Sample']])]
            ## get pGL0
            output_table['Strain'].append(cvm)
            output_table['ORI (pGL0)'].append(p0)
            output_table['Plasmid (pGL2)'].append(p2)
            output_table['Fold enrichment'].append(row['value'])
    ## get pGL2
output_table = pd.DataFrame(output_table)
output_table.to_csv("ingest_for_portal147.csv")

In [113]:
output_table.head(output_table.shape[0] - 250).to_csv("ingest_for_portal150_1.csv")

In [114]:
output_table.tail(250).to_csv("ingest_for_portal150_2.csv")

In [100]:
output_table

,Strain,ORI (pGL0),Plasmid (pGL2),Fold enrichment
0,CVM007,pGL0_16 [2μ],pGL2_209,0.0
1,CVM007,pGL0_16 [2μ],pGL2_209,0.0
2,CVM007,pGL0_16 [2μ],pGL2_209,0.0
3,CVM010,pGL0_16 [2μ],pGL2_209,0.0
4,CVM010,pGL0_16 [2μ],pGL2_209,0.0
...,...,...,...,...
523,CVM027,pGL0_20 [panARS-opt],pGL2_208,0.0
524,CVM027,pGL0_20 [panARS-opt],pGL2_208,0.0
525,CVM036,pGL0_20 [panARS-opt],pGL2_208,0.0
526,CVM036,pGL0_20 [panARS-opt],pGL2_208,0.0


In [97]:
250+278

528